In [1]:
%%html
<style> body { font-family: "Times New Roman"} </style>   

<center>
    <br>
    <span style="color: #001a33; font-size: 1.6em;">ANÁLISIS EXPLORATORIO DE METADATOS DE LOS PAQUETES R ALMACENADOS EN CRAN</span>
    <br>
    <br>
</center>

1. <a href='#section1.0'>Introducción</a><br/>
2. <a href='#section2.0'>Carga de datos</a><br/>
3. <a href='#section3.0'>Preparación de los datos</a><br/>
4. <a href='#section4.0'>Exploración de datos</a><br/>

<a id='section1.0'></a>
# 1. Introducción

El conjunto de datos fue extraído utilizando una API de acceso a los metadatos de la base de datos de CRAN que se encuentra en el siguiente repositorio https://github.com/r-hub/crandb. El código utilizado para la extracción está en un Script R, y tiene la funcionalidad de guardar los datos extraídos en una base de datos MongoDB.

A continuación se describen algunas de las variables de nuestro conjunto de datos:

- **Package**: Nombre del paquete
- **Type**: Tipo de paquete, generalmente “Package”
- **Title**: Es una descripción de una línea del paquete
- **Version**: Versión del paquete
- **Date**: Es un valor en formato fecha, no se conoce que representa
- **Author**: Identifica al autor o autores del paquete
- **Maintainer**: Identifica al mantenedor del paquete
- **Description**: Es un descripción más extensa del paquete
- **License**: Especifica la licencia del paquete, generalmente suele ser una abreviatura estándar de una licencia de código abierto, como MIT, GPL-2 o BSD
- **Depends**: Especifica la versión de R que requiere el paquete. Pero antes del despliegue de namespaces en R 2.14.0 en este campo se especificaban las dependencias a otros paquetes
- **Suggests**: Especifica los paquetes que puede usar el paquete, pero no son necesariamente requeridos
- **Imports**: Especifica los paquetes que deben estar presentes para que el pa-quete funcione
- **NeedsCompilation**: Si el paquete necesita compilación
- **Packaged**: Fecha de empaquetado
- **Repository**: Nombre del repositorio
- **Date/Publication**: Fecha de publicación del paquete
- **crandb_file_date**: Fecha en la que se actualizó el fichero de descripción del paquete

In [2]:
from platform import python_version
print(python_version())

3.7.3


In [3]:
from pandas.io.json import json_normalize
from bson import json_util, ObjectId
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymongo
import json
import re

<a id='section2.0'></a>
# 2. Carga de datos

Conexión a la base de datos donde están los metadatos de los paquetes R del repositorio CRAN

In [4]:
mongo = pymongo.MongoClient("mongodb://localhost:27017/")
database = mongo["DB_CRAN"]
collection = database["R_packages"]

Extración de documentos JSON

In [5]:
collection_packages = list(collection.find({}, {'_id': False}))

packages = []
for c in collection_packages:
    packages.append(json.loads(json_util.dumps(c)))
      
print(f'HAY {len(packages)} DOCUMENTOS')

HAY 14890 DOCUMENTOS


Cada documento contiene metadatos de un paquete. Para cada uno se especifica el nombre del paquete, descripción, última versión del paquete, y las versiones del paquete como un subdocumento con información de cada versión.

In [6]:
packages[0]

{'A3': {'_id': ['A3'],
  '_rev': ['5-b7e2fc1ff5bb5e68d8eafa1d948d9e34'],
  'name': ['A3'],
  'versions': {'0_9_1': {'Package': ['A3'],
    'Type': ['Package'],
    'Title': ['A3: Accurate, Adaptable, and Accessible Error Metrics for<U+000a>Predictive Models'],
    'Version': ['0.9.1'],
    'Date': ['2013-02-06'],
    'Author': ['Scott Fortmann-Roe'],
    'Maintainer': ['Scott Fortmann-Roe <scottfr@berkeley.edu>'],
    'Description': ['This package supplies tools for tabulating and analyzing<U+000a>the results of predictive models. The methods employed are<U+000a>applicable to virtually any predictive model and make<U+000a>comparisons between different methodologies straightforward.'],
    'License': ['GPL (>= 2)'],
    'Depends': {'xtable': ['*'], 'pbapply': ['*']},
    'Suggests': {'randomForest': ['*'], 'e1071': ['*']},
    'Packaged': ['2013-02-06 16:46:12 UTC; scott'],
    'Repository': ['CRAN'],
    'Date/Publication': ['2013-02-07 10:00:27'],
    'crandb_file_date': ['2013-02-07 

<a id='section3.0'></a>
# 3. Preparación de los datos

Se extrae en una lista todos los nombres de los paquetes

In [7]:
list_pkgs = []
for i in packages:
    for k, v in i.items():
        list_pkgs.append(k)
        
print(list_pkgs[0:20])

['A3', 'aaSEA', 'abbyyR', 'abc', 'abc_data', 'ABC_RAP', 'ABCanalysis', 'abcdeFBA', 'ABCoptim', 'ABCp2', 'abcrf', 'abctools', 'abd', 'abe', 'abf2', 'ABHgenotypeR', 'abind', 'abjutils', 'abn', 'abnormality']


A continuación se extraen el subdocumento "versions" de cada paquete, el mismo contiene la información de las versiones del paquete

In [8]:
versions = []
con = 0

for i in list_pkgs:
    if (con <= len(packages)):
        versions.append(packages[con][i]['versions'])
        con = con + 1
#versions[0]

Creamos en un diccionario de listas con la información de todas las versiones de cada paquete

In [9]:
versions_pkgs = []

for i in versions:
    for k, v in i.items():
        versions_pkgs.append(v)

versions_pkgs[0]

{'Package': ['A3'],
 'Type': ['Package'],
 'Title': ['A3: Accurate, Adaptable, and Accessible Error Metrics for<U+000a>Predictive Models'],
 'Version': ['0.9.1'],
 'Date': ['2013-02-06'],
 'Author': ['Scott Fortmann-Roe'],
 'Maintainer': ['Scott Fortmann-Roe <scottfr@berkeley.edu>'],
 'Description': ['This package supplies tools for tabulating and analyzing<U+000a>the results of predictive models. The methods employed are<U+000a>applicable to virtually any predictive model and make<U+000a>comparisons between different methodologies straightforward.'],
 'License': ['GPL (>= 2)'],
 'Depends': {'xtable': ['*'], 'pbapply': ['*']},
 'Suggests': {'randomForest': ['*'], 'e1071': ['*']},
 'Packaged': ['2013-02-06 16:46:12 UTC; scott'],
 'Repository': ['CRAN'],
 'Date/Publication': ['2013-02-07 10:00:27'],
 'crandb_file_date': ['2013-02-07 04:00:29'],
 'date': ['2013-02-07T10:00:27+00:00'],
 'releases': ['2.15.3']}

Como se puede observar el valor de cada clave es una lista de un solo elemento, y en otras la clave es un diccionario con listas. El siguiente fragmento de código se utiliza para tratar estos datos, dejando para cada clave un valor *String* o *Diccionario*. Además extraemos y añadimos a cada paquete la versión de R si se especifica

In [10]:
results = []

for i in versions_pkgs:
    pkg = {}
    for k, v in i.items():
        if isinstance(v, dict):
            value = [l for l in v ]
            version = [''.join(l) for l in v.values() ]
            
            pkg[k] = value
            
            for version_R, version in zip(value, version):
                if version_R == "R":
                    pkg.update({ "Version_" + version_R :''.join(version) })
                    
        elif isinstance(v, list):
            if len(v) <= 1 and k != "releases":
                pkg[k] = ''.join(v)
            else:
                pkg[k] = v
    results.append(json.loads(json_util.dumps(pkg)))

In [11]:
results[0]

{'Package': 'A3',
 'Type': 'Package',
 'Title': 'A3: Accurate, Adaptable, and Accessible Error Metrics for<U+000a>Predictive Models',
 'Version': '0.9.1',
 'Date': '2013-02-06',
 'Author': 'Scott Fortmann-Roe',
 'Maintainer': 'Scott Fortmann-Roe <scottfr@berkeley.edu>',
 'Description': 'This package supplies tools for tabulating and analyzing<U+000a>the results of predictive models. The methods employed are<U+000a>applicable to virtually any predictive model and make<U+000a>comparisons between different methodologies straightforward.',
 'License': 'GPL (>= 2)',
 'Depends': ['xtable', 'pbapply'],
 'Suggests': ['randomForest', 'e1071'],
 'Packaged': '2013-02-06 16:46:12 UTC; scott',
 'Repository': 'CRAN',
 'Date/Publication': '2013-02-07 10:00:27',
 'crandb_file_date': '2013-02-07 04:00:29',
 'date': '2013-02-07T10:00:27+00:00',
 'releases': ['2.15.3']}

Tabulamos los documentos JSON, utilizando la función json_normalize

In [12]:
df = pd.DataFrame(json_normalize(results))

In [13]:
df.head()

,Package,Type,Title,Version,Date,Author,Maintainer,Description,License,Depends,...,Twitter,Co-author,Saveimage,SystemRequirement,Versionnote,Classification_ACM,ReleaseDate,xxxLazyLoad,Github URL,Github
0,A3,Package,"A3: Accurate, Adaptable, and Accessible Error ...",0.9.1,2013-02-06,Scott Fortmann-Roe,Scott Fortmann-Roe <scottfr@berkeley.edu>,This package supplies tools for tabulating and...,GPL (>= 2),"[xtable, pbapply]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A3,Package,"A3: Accurate, Adaptable, and Accessible Error ...",0.9.2,2013-03-24,Scott Fortmann-Roe,Scott Fortmann-Roe <scottfr@berkeley.edu>,This package supplies tools for tabulating and...,GPL (>= 2),"[R, xtable, pbapply]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3,Package,"Accurate, Adaptable, and Accessible Error Metr...",1.0.0,2015-08-15,Scott Fortmann-Roe,Scott Fortmann-Roe <scottfr@berkeley.edu>,Supplies tools for tabulating and analyzing th...,GPL (>= 2),"[R, xtable, pbapply]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aaSEA,Package,Amino Acid Substitution Effect Analyser,1.0.0,NaN,Raja Sekhara Reddy D.M,Raja Sekhara Reddy D.M <raja.duvvuru@gmail.com>,"Given a protein multiple sequence alignment, i...",GPL-3,[R],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abbyyR,NaN,Access to Abbyy Optical Character Recognition ...,0.1,NaN,"Gaurav Sood [aut, cre]",Gaurav Sood <gsood07@gmail.com>,Get text from images of text using Abbyy Cloud...,GPL (>= 2),[R],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<a id='section4.0'></a>
# 4. Limpieza de datos

Algunos paquetes tienen más de 200 campos, y cuando se tabulan al no existir estos campos en las mayoría de los paquetes se han generado varios valores *NaN*.

In [14]:
df.shape

(87912, 225)

In [15]:
df.isna().sum().to_frame().head(20)

,0
Package,0
Type,40355
Title,157
Version,19
Date,21335
Author,33
Maintainer,224
Description,81
License,29
Depends,16663


Lista de columnas en el dataframe

In [16]:
print(df.columns.tolist())

['Package', 'Type', 'Title', 'Version', 'Date', 'Author', 'Maintainer', 'Description', 'License', 'Depends', 'Suggests', 'Packaged', 'Repository', 'Date/Publication', 'crandb_file_date', 'date', 'releases', 'Version_R', 'NeedsCompilation', 'Imports', 'Encoding', 'LazyData', 'RoxygenNote', 'VignetteBuilder', 'MD5sum', 'Authors@R', 'URL', 'BugReports', 'Contact', 'SystemRequirements', 'LazyLoad', 'Lazyload', 'Classification/ACM', 'Classification/JEL', 'LinkingTo', 'Repository/R-Forge/Project', 'Repository/R-Forge/Revision', 'Collate', 'BuildVignettes', 'Additional_repositories', 'Repository/R-Forge/DateTimeStamp', 'Keywords', 'Authors', 'biocViews', 'ByteCompile', 'ZipData', 'Url', 'Enhances', 'Copyright', 'Revision', 'Publication/Date', 'Developpers', 'Contributed datasets from', 'Language', 'Requires', 'RdMacros', 'Suggest', 'License_restricts_use', 'X-CRAN-Original-Maintainer', 'X-CRAN-Original-URL', 'X-CRAN-Comment', 'RcppModules', 'SaveImage', 'References', 'PatchURL', 'SuggestsNote

Se establece un umbral 1000 para mantener solo las columnas que tengan al menos mil valores que no sean NaN

In [17]:
df = df.dropna(axis=1, thresh=1000)
df.shape

(87912, 39)

Lista de columnas

In [18]:
print(df.columns.tolist())

['Package', 'Type', 'Title', 'Version', 'Date', 'Author', 'Maintainer', 'Description', 'License', 'Depends', 'Suggests', 'Packaged', 'Repository', 'Date/Publication', 'crandb_file_date', 'date', 'releases', 'Version_R', 'NeedsCompilation', 'Imports', 'Encoding', 'LazyData', 'RoxygenNote', 'VignetteBuilder', 'MD5sum', 'Authors@R', 'URL', 'BugReports', 'SystemRequirements', 'LazyLoad', 'LinkingTo', 'Repository/R-Forge/Project', 'Repository/R-Forge/Revision', 'Collate', 'Repository/R-Forge/DateTimeStamp', 'ByteCompile', 'Enhances', 'Copyright', 'Priority']


In [19]:
df.columns.tolist()

['Package',
 'Type',
 'Title',
 'Version',
 'Date',
 'Author',
 'Maintainer',
 'Description',
 'License',
 'Depends',
 'Suggests',
 'Packaged',
 'Repository',
 'Date/Publication',
 'crandb_file_date',
 'date',
 'releases',
 'Version_R',
 'NeedsCompilation',
 'Imports',
 'Encoding',
 'LazyData',
 'RoxygenNote',
 'VignetteBuilder',
 'MD5sum',
 'Authors@R',
 'URL',
 'BugReports',
 'SystemRequirements',
 'LazyLoad',
 'LinkingTo',
 'Repository/R-Forge/Project',
 'Repository/R-Forge/Revision',
 'Collate',
 'Repository/R-Forge/DateTimeStamp',
 'ByteCompile',
 'Enhances',
 'Copyright',
 'Priority']

Siendo las columnas Imports, Depends y Suggests una lista de valores que especifica los paquetes de los que depende, entonces podemos obtener el tamaño de cada lista y así determinar el número de dependencias para cada categoría de dependencia

In [20]:
columns = ['Author','Imports','Depends','Suggests','LinkingTo']

for col in columns:
    df.loc[df[col].isnull(),[col]] = df.loc[df[col].isnull(), col].apply(lambda x: [])

In [21]:
df['NumberOfImports'] = df['Imports'].str.len()
df['NumberOfDepends'] = df['Depends'].str.len()
df['NumberOfSuggests'] = df['Suggests'].str.len()
df['NumberOfLinkingTo'] = df['LinkingTo'].str.len()

Si la columna Fecha de Publicación es nula, la completamos con la columna Fecha

In [22]:
df.loc[df['Date/Publication'].isna(),'Date/Publication'] = df['date']

Subdividimos la fecha de publicación par obtener el año, mes y día de la publicación del paquete

In [23]:
df['DatePublication'] = pd.to_datetime(df['Date/Publication'], utc=True).values.astype('M8[D]')
df['Year/Publication'] = df['DatePublication'].dt.strftime('%Y')
df['Month/Publication'] = df['DatePublication'].dt.strftime('%m')
df['Day/Publication'] = df['DatePublication'].dt.strftime('%d')
df["Year_Month/Publication"] = df['DatePublication'].dt.strftime('%Y-%m')

In [24]:
df.head(3)

,Package,Type,Title,Version,Date,Author,Maintainer,Description,License,Depends,...,Priority,NumberOfImports,NumberOfDepends,NumberOfSuggests,NumberOfLinkingTo,DatePublication,Year/Publication,Month/Publication,Day/Publication,Year_Month/Publication
0,A3,Package,"A3: Accurate, Adaptable, and Accessible Error ...",0.9.1,2013-02-06,Scott Fortmann-Roe,Scott Fortmann-Roe <scottfr@berkeley.edu>,This package supplies tools for tabulating and...,GPL (>= 2),"[xtable, pbapply]",...,NaN,0,2,2,0,2013-02-07,2013,02,07,2013-02
1,A3,Package,"A3: Accurate, Adaptable, and Accessible Error ...",0.9.2,2013-03-24,Scott Fortmann-Roe,Scott Fortmann-Roe <scottfr@berkeley.edu>,This package supplies tools for tabulating and...,GPL (>= 2),"[R, xtable, pbapply]",...,NaN,0,3,2,0,2013-03-26,2013,03,26,2013-03
2,A3,Package,"Accurate, Adaptable, and Accessible Error Metr...",1.0.0,2015-08-15,Scott Fortmann-Roe,Scott Fortmann-Roe <scottfr@berkeley.edu>,Supplies tools for tabulating and analyzing th...,GPL (>= 2),"[R, xtable, pbapply]",...,NaN,0,3,2,0,2015-08-16,2015,08,16,2015-08


In [25]:
colnames = df.columns.tolist()

In [26]:
df.to_csv("DATA_CRAN/CRAN_DATA.csv", header=colnames, sep="|", index=False)

In [27]:
df.dtypes

Package                                     object
Type                                        object
Title                                       object
Version                                     object
Date                                        object
Author                                      object
Maintainer                                  object
Description                                 object
License                                     object
Depends                                     object
Suggests                                    object
Packaged                                    object
Repository                                  object
Date/Publication                            object
crandb_file_date                            object
date                                        object
releases                                    object
Version_R                                   object
NeedsCompilation                            object
Imports                        